In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Reading dataset

In [79]:
dataset=np.genfromtxt("C:\\Users\Dharmik\Desktop\data.csv",delimiter=",")

x=dataset[1:,1:-1] #shape(1000,28)

y_raw=dataset[1:,-1]

y=y_raw.reshape((1000,1)) #shape(1000,1)

y=y.astype('int')

x_train,x_test,y_train,y_test=train_test_split(x, y, test_size=0.25, random_state=112)

len_y=len(y_train)



In [4]:
print(x_train.shape)
print(y_train.shape)

(750, 28)
(750, 1)


In [156]:
epochs=10000
learning_rate=0.
hiddenlayer_num=20
n=x.shape[0]


# Sigmoid and its derivative


In [157]:
def sigmoid(z):
    return (1/(1+np.exp(-z)))

def sigmoidDerivative(sigmoid):
    return sigmoid * (1 - sigmoid)


# Initializing weights 

In [158]:

W1=np.random.randn(hiddenlayer_num,x_train.shape[1])* (np.sqrt(2/(28*20)))#shape(20,28)
#b1=np.random.rand(x_train.shape[0],1)
W2=np.random.randn(10,hiddenlayer_num)*(np.sqrt(2/(20*10))) #shape(10,20) 10 is the number of classes
#b2=np.random.rand(x_train.shape[0],1)
print(W1.shape) 
#print(b1.shape)
print(W2.shape)
#print(b2.shape)

(20, 28)
(10, 20)


# Feedforward

In [159]:
def feedforward(x_train,W1,W2):
    A1 = sigmoid(np.dot(W1,x_train.T))     #shape(20,1000)
    A2 = sigmoid(np.dot(W2,A1)).T          #shape(1000,10)
    print(A1.shape)
    print(A2.shape)
    return A2,A1

A2,A1 = feedforward(x_train,W1,W2) #defining variables for the output of feedforward network which will be used in backpropagation


(20, 750)
(750, 10)


c:\users\dharmik\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


# One hot Encoder

In [160]:
def onehotencoder(onehot_raw):
    onehot_raw=np.zeros((x_train.shape[0],10))# to create a matrix of zeros which will provide a frameworkto create one hot encoded matrix of y
    for i in range(len_y):
        onehot_raw[i][int(y[i])]=1  
    return onehot_raw   #creates a onehotencoded matrix of (1000,10) shape

print(onehotencoder(y_train), onehotencoder(y_train).shape,sep='\n \n')
#at first the onehotencoded matrix was not coming out to be proper. that is because the y matrix had dtype of float and not int. so when I type converted it into int dtpye the proper one hot encoded matrix was obtained.


[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
 
(750, 10)


# Backpropagation

In [161]:
Y=onehotencoder(y_train) #onehotencoded y matrix

def backpropagation(W2,W1,A2,A1,x,Y):
    for iters in range(epochs):
        Z1 = np.dot(x_train, W1.T)      #shape(1000,20)
        A1 = sigmoid(Z1)        #shape(1000,20)
        Z2 = np.dot(A1, W2.T)     #shape(1000,10)
        A2 = sigmoid(Z2)          #shape(1000,10)
        cost = np.sum((Y * np.log(A2)) + ((1-Y) * np.log(1-A2))) * (-1/n)
        
        #calculating derivatives of Z2,W2,A1,W1 wrt cost function
        
        dZ2=(-1/n)*(Y-A2)                #shape(1000,10)
        dW2=np.dot(dZ2.T, A1)            #shape(10,20)
        dA1=np.dot(dZ2,W2)               #shape(1000,20)
        dZ1=   dA1*sigmoidDerivative(A1) #shape(1000, 20)    ####################
        dW1=np.dot(x_train.T,dZ1).T      #shape(28,10)
        
        #updating weights 
        
        W2=W2-learning_rate*dW2
        W1=W1-learning_rate*dW1
        
    
    return W2,W1
W2,W1=backpropagation(W2,W1,A2,A1,x,Y)

print(W2.shape)
print(W1.shape)

c:\users\dharmik\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


(10, 20)
(20, 28)


# Test Function

In [162]:
def test(x,y,W1,W2):
    Z1 = np.dot(x_test,W1.T) 
    A1 = sigmoid(Z1) 
    Z2 = np.dot(A1,W2.T)
    A2 = sigmoid(Z2)
    acc=0
    #print(A2.shape, A2[0].astype('float64'))
    predicted=np.argmax(A2,axis=1)
    print(A2)
    print(predicted)
    #label=np.argmax(y_test,axis=0)
    #print(label)
    #print(label.shape)
    for i in range(len(y_test)):
        if(predicted[i]==y[i]):
            acc=acc+1
    accuracy= acc/len(y_test)
    print(accuracy)
    return predicted

p = test(x_test,y_test,W1,W2)


    

[[0.47365802 0.51695474 0.60358931 ... 0.52639452 0.40067227 0.48918781]
 [0.46832505 0.56800869 0.61639258 ... 0.50341056 0.43796302 0.53441255]
 [0.4736586  0.51695465 0.60358983 ... 0.52639409 0.40067199 0.48918911]
 ...
 [0.46585518 0.57403314 0.61474397 ... 0.50160123 0.44096693 0.5366894 ]
 [0.46837735 0.56788087 0.61642743 ... 0.50344889 0.43789943 0.53436428]
 [0.46789854 0.56905013 0.61610811 ... 0.50309826 0.43848121 0.53480562]]
[5 2 5 5 2 2 2 2 5 5 2 5 5 2 2 5 2 2 2 2 2 5 5 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 5 2 2 2 2 2 2 5 5 2 5 2 2 5 5 5 2 2 2 5 2 2 2 2 2 2 5 2 2 2 2 2 2 2 5
 2 2 2 5 2 2 5 2 2 5 5 5 5 2 5 2 2 5 2 2 2 5 2 2 2 2 5 2 5 2 2 5 2 2 2 2 2
 2 2 2 5 5 2 2 5 2 2 2 2 2 5 2 2 5 5 2 5 2 2 2 5 2 5 2 2 5 5 5 2 2 2 2 2 2
 2 2 2 2 2 5 2 2 2 2 2 5 5 2 2 2 2 2 2 5 5 2 2 2 2 2 2 5 2 2 2 5 2 5 5 5 5
 5 2 5 5 2 2 5 2 5 2 2 2 5 5 5 2 5 2 5 5 2 2 2 2 2 2 5 2 5 2 2 2 2 2 2 2 2
 5 5 2 5 2 2 2 5 5 2 2 2 2 2 2 2 5 5 5 5 5 5 2 2 2 2 2 2]
0.104


c:\users\dharmik\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [163]:
p

array([5, 2, 5, 5, 2, 2, 2, 2, 5, 5, 2, 5, 5, 2, 2, 5, 2, 2, 2, 2, 2, 5,
       5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 2, 2, 2, 2,
       2, 2, 5, 5, 2, 5, 2, 2, 5, 5, 5, 2, 2, 2, 5, 2, 2, 2, 2, 2, 2, 5,
       2, 2, 2, 2, 2, 2, 2, 5, 2, 2, 2, 5, 2, 2, 5, 2, 2, 5, 5, 5, 5, 2,
       5, 2, 2, 5, 2, 2, 2, 5, 2, 2, 2, 2, 5, 2, 5, 2, 2, 5, 2, 2, 2, 2,
       2, 2, 2, 2, 5, 5, 2, 2, 5, 2, 2, 2, 2, 2, 5, 2, 2, 5, 5, 2, 5, 2,
       2, 2, 5, 2, 5, 2, 2, 5, 5, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5,
       2, 2, 2, 2, 2, 5, 5, 2, 2, 2, 2, 2, 2, 5, 5, 2, 2, 2, 2, 2, 2, 5,
       2, 2, 2, 5, 2, 5, 5, 5, 5, 5, 2, 5, 5, 2, 2, 5, 2, 5, 2, 2, 2, 5,
       5, 5, 2, 5, 2, 5, 5, 2, 2, 2, 2, 2, 2, 5, 2, 5, 2, 2, 2, 2, 2, 2,
       2, 2, 5, 5, 2, 5, 2, 2, 2, 5, 5, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5,
       5, 5, 2, 2, 2, 2, 2, 2], dtype=int32)

In [164]:
print(classification_report(y_test, p))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        25
           1       0.00      0.00      0.00        27
           2       0.08      0.57      0.13        23
           3       0.00      0.00      0.00        25
           4       0.00      0.00      0.00        26
           5       0.16      0.48      0.25        27
           6       0.00      0.00      0.00        28
           7       0.00      0.00      0.00        23
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00        26

   micro avg       0.10      0.10      0.10       250
   macro avg       0.02      0.10      0.04       250
weighted avg       0.02      0.10      0.04       250

